In [1]:
from cobra.io import load_json_model
from os import getcwd
from Functions_Modules import curation_tools as ct
from math import isnan
from numpy import zeros
from numpy import isnan

In [2]:
relative_directory = getcwd()
model = load_json_model(relative_directory + '/Reconstructions/MethylococcusModel14.json')

# Basic Information

## Metabolites No Charge

In [3]:
[met for met in model.metabolites if met.charge == None]

[<Metabolite lipa_cold_e at 0x10ed6f8d0>,
 <Metabolite lipa_cold_p at 0x10ed6f910>,
 <Metabolite eca4colipa_e at 0x10ed6f950>,
 <Metabolite eca4colipa_p at 0x10ed6f990>,
 <Metabolite enlipa_p at 0x10ed6f9d0>,
 <Metabolite enlipa_e at 0x10ed6fa10>,
 <Metabolite o16a4colipa_e at 0x10ed6fa50>,
 <Metabolite o16a4colipa_p at 0x10ed6fa90>,
 <Metabolite no3_e at 0x10ed6fd10>]

In [4]:
model.metabolites.lipa_cold_p.charge = 0
model.metabolites.lipa_cold_e.charge = 0
model.metabolites.eca4colipa_e.charge = 0
model.metabolites.eca4colipa_p.charge  = 0
model.metabolites.enlipa_e.charge = 0
model.metabolites.enlipa_p.charge = 0
model.metabolites.o16a4colipa_e.charge = 0
model.metabolites.o16a4colipa_p.charge = 0
model.metabolites.no3_e.charge = model.metabolites.no3_p.charge

In [5]:
[met for met in model.metabolites if met.charge == None]

[]

## Reactions No GPR

In [6]:
for rxn in model.reactions: 
    if rxn.gene_reaction_rule == '':
            if not 'EX_' in rxn.id:
                if not 'ex' in rxn.id:
                    if not 'DM_' in rxn.id:
                        if not '_GAPFILLING' in rxn.id:
                            print rxn.id

TKT1
ATPM
PROD2
BIOMASS_Mcapsulatus
MEOHtrpp
CH4trpp
FALDtpp
ALDD1
H2Otpp
O2tpp
FACOAL180
FACOAL140
DESAT14_Modified
FACOAE141
FA180ACPHi
FA160ACPHi
FA140ACPHi
FA150ACPHi
FA170ACPHi
FACOAEcyc170i
SMO
LNSTLS
PMDPHT
PPPGO
CPC6R
MC_Average_FattyAcid
MC_AFAA
CLPNS_MC
GLYCDx
PPTGS_MC
BPNT
HAMtpp
NO2tpp
PFK_3_ppi
CO2t
COt
N2trpp


In [7]:
model.reactions.PFK_3_ppi.gene_reaction_rule = model.reactions.PFK_ppi.gene_reaction_rule

In [8]:
model.reactions.TKT1.gene_reaction_rule = 'MCA3040 or MCA3046'

In [9]:
model.reactions.FACOAL140.gene_reaction_rule = model.reactions.FACOAL160.gene_reaction_rule
model.reactions.FACOAL180.gene_reaction_rule = model.reactions.FACOAL160.gene_reaction_rule

In [10]:
model.reactions.FACOAE141.gene_reaction_rule = model.reactions.FACOAE160.gene_reaction_rule

In [11]:
model.reactions.FACOAE160.gene_reaction_rule

u'MCA1401 or MCA1006 or MCA1569'

In [12]:
model.reactions.BPNT.gene_reaction_rule = 'MCA2983'

# Stoichiomentric Consistency

## Imbalanced Reactions

In [13]:
ct.superprint(model,'PAMMOipp')

nh4_p + o2_p + q8h2_im --> h2o_p + ham_p + q8_im 

q8h2_im C49H76O4 0
o2_p O2 0
nh4_p H4N 1
<=>
h2o_p H2O 0
q8_im C49H74O4 0
ham_p H3NO 0


In [14]:
model.reactions.PAMMOipp.check_mass_balance()

{'H': -1.0, 'charge': -1.0}

In [15]:
model.reactions.PAMMOipp.add_metabolites({model.metabolites.h_p:1}, combine= True)

In [16]:
ct.superprint(model,'NO3t2pp')

atp_c + h2o_c + h_p + no3_p --> adp_c + h_c + no3_c + pi_c 

atp_c C10H12N5O13P3 -4
no3_p NO3 -1
h_p H 1
h2o_c H2O 0
<=>
h_c H 1
no3_c NO3 -1
pi_c HO4P -2
adp_c C10H12N5O10P2 -3


In [17]:
model.reactions.NO3t2pp.check_mass_balance()

{'H': -1, 'charge': -1}

In [18]:
model.reactions.NO3t2pp.add_metabolites({model.metabolites.h_p:1}, combine= True)

In [19]:
ct.superprint(model,'NO3tex')

no3_e --> no3_p 

no3_e NO3 -1
<=>
no3_p NO3 -1


In [20]:
model.reactions.NO3tex.check_mass_balance()

{}

In [21]:
ct.superprint(model,'PMMODCipp')

ch4_p + 2 focytcc555_p + o2_p --> 2 ficytcc555_p + h2o_p + meoh_p 

ch4_p CH4 0
o2_p O2 0
focytcc555_p C42H54FeN8O6S2 2
<=>
h2o_p H2O 0
meoh_p CH4O1 0
ficytcc555_p C42H54FeN8O6S2 3


In [22]:
model.reactions.PMMODCipp.check_mass_balance()

{'H': 2.0, 'charge': 2.0}

In [23]:
model.reactions.PMMODCipp.add_metabolites({model.metabolites.h_p:-2}, combine= True)

# Syntax Information

## Compartments

In [24]:
len(model.reactions)

896

In [25]:
model.compartments.pop('om')

u'outer membrane'

In [26]:
model.compartments.pop('bm')

u'biomass'

In [27]:
len(model.reactions)

896

## Untagged ABC Transport

In [28]:
model.reactions.NO3t2pp

Reaction identifier,NO3t2pp
Name,Nitrate transport in via proton symport (periplasm)
Memory address,0x010f0dc150
Stoichiometry,atp_c + h2o_c + no3_p --> adp_c + h_c + no3_c + pi_c ATP + H2O + nitrate --> ADP + H(+) + nitrate + phosphate
GPR,MCA0589 or MCA0594
Lower bound,0.0
Upper bound,1000.0


In [29]:
model.reactions.NO3t2pp.add_metabolites({model.metabolites.h_p:-1,
                                         model.metabolites.atp_c:1,
                                         model.metabolites.adp_c:-1,
                                         model.metabolites.pi_c:-1,
                                         model.metabolites.h2o_c:1},combine=True)

In [30]:
model.reactions.NO3t2pp

Reaction identifier,NO3t2pp
Name,Nitrate transport in via proton symport (periplasm)
Memory address,0x010f0dc150
Stoichiometry,h_p + no3_p --> h_c + no3_c H(+) + nitrate --> H(+) + nitrate
GPR,MCA0589 or MCA0594
Lower bound,0.0
Upper bound,1000.0


## Upper Case Metabolites

In [31]:
model.metabolites.Glc_aD_c.id = 'glc__aD_c'

In [32]:
model.metabolites.phyQ_c.reactions

frozenset({<Reaction DMTPHT at 0x10eef9b50>})

In [33]:
model.reactions.DMTPHT

Reaction identifier,DMTPHT
Name,Demethylphylloquinone methyltransferase
Memory address,0x010eef9b50
Stoichiometry,ahcys_c + h_c + phyQ_c --> amet_c + dmtphllqne_c S-adenosyl-L-homocysteine + H(+) + Phylloquinone --> S-adenosyl-L-methionine + demethylphylloquinone
GPR,MCA2017
Lower bound,0.0
Upper bound,1000.0


In [34]:
model.metabolites.Ncbmpts_c.remove_from_model()

In [35]:
model.metabolites.C04051_c.reactions

frozenset({<Reaction ADPT2 at 0x10ef1df10>})

In [36]:
model.remove_reactions([model.reactions.ADPT2])

In [37]:
model.metabolites.C04051_c.remove_from_model()

In [38]:
model.metabolites.lipidAds_c.reactions

frozenset({<Reaction TDSK at 0x10efb1f90>, <Reaction LPADSS at 0x10efc30d0>})

In [39]:
model.metabolites.get_by_id("15HPET_c").remove_from_model()

In [40]:
model.metabolites.get_by_id("5HPET_c").remove_from_model()

In [41]:
model.metabolites.leuktrC4_c.remove_from_model()

In [42]:
model.metabolites.leuktrD4_c.remove_from_model()

In [43]:
model.metabolites.focytC_c.remove_from_model()

In [44]:
model.metabolites.ficytC_c.remove_from_model()

# Model ID

In [45]:
model.id = 'iCL730'

In [46]:
model.reactions.PMMOipp.subsystem

u'rn00680:Methane metabolism,rn01100:Metabolic pathways,rn01120:Microbial metabolism in diverse environments,rn01200:Carbon metabolism'

# Biomass

## Precursors that cannot be produced

In [47]:
for met in ['adocbl_c', 'agdpcbi_c', 'adocbip_c', 'applp_c', 'appl_c', 'adocbi_c', 'adcobhex_c', 
            'adcobdam_c','co1dam_c', 'co2dam_c', 'cobya_c', 'codhpre6_c','copre6_c', 'copre5_c',
            'amet_c', 'ahcys_c', 'acald_c',
            'copre4_c', 'copre3_c', 'copre2_c', 'dscl_c', 'uppg3_c', 'hmbil_c', 'ppbng_c', '5aop_c',
            'rdmbzi_c', '5prdmbz_c', 'dmbzid_c', 'fmnh2_c','fmn_c']:
        with model:
            for exchange in model.exchanges:
                exchange.bounds = -1000, 1000
            model.add_boundary(model.metabolites.get_by_id(met))
            model.objective = model.reactions.get_by_id('EX_'+met)
            solution = model.optimize()
            print met, solution.f

adocbl_c 4.4408920985e-16
agdpcbi_c 1.58603289232e-16
adocbip_c 0.0
applp_c 769.963085399
appl_c 0.0
adocbi_c 0.0
adcobhex_c 0.0
adcobdam_c 0.0
co1dam_c 0.0
co2dam_c -1.91583774419e-48
cobya_c 1.64346021921e-31
codhpre6_c 3.28692043842e-32
copre6_c -5.99862980012e-31
copre5_c 62.3134269105
amet_c 86.4936349028
ahcys_c 0.0
acald_c 0.0
copre4_c 64.3291276347
copre3_c 66.2790830946
copre2_c 68.5642121685
dscl_c 69.2120629921
uppg3_c 74.1391025641
hmbil_c 76.55401498
ppbng_c 296.556410256
5aop_c 789.522649573
rdmbzi_c 1000.0
5prdmbz_c 1000.0
dmbzid_c 102.445734678
fmnh2_c 87.1800923226
fmn_c 85.9275047893


In [48]:
model.add_boundary(model.metabolites.acald_c, type='demand', lb=0, ub=1000)
model.reactions.DM_acald_c.name = 'Demand needed to allow acetaldehyde to leave system'
model.add_boundary(model.metabolites.pppi_c, type='demand', lb=0, ub=1000)
model.reactions.DM_pppi_c.name = 'Demand needed to allow pppi to leave system'
model.add_boundary(model.metabolites.gmp_c, type='demand', lb=0, ub=1000)
model.reactions.DM_gmp_c.name = 'Demand needed to allow gmp to leave system'

I ran the two cells above repeatedly and figured out that 3 reactions produced metabolites which cannot be 'recycled' we lack information on degradation pathways. Hence, three demand reactions were added to allow these metabolites to leave the model.

In [33]:
model.optimize()

UnicodeEncodeError: 'ascii' codec can't encode character u'\xd7' in position 6397: ordinal not in range(128)

<Solution 0.088 at 0x11503af10>

In [22]:
[met for met in model.metabolites if met.id.endswith("_im")]

[<Metabolite q8_im at 0x114a19e90>, <Metabolite q8h2_im at 0x114a19ed0>]

In [23]:
model.metabolites.q8_im.id = "q8_c"

In [24]:
model.metabolites.q8h2_im.id = "q8h2_c"

In [25]:
[met for met in model.metabolites if met.id.endswith("_im")]

[]

In [36]:
model.compartments.pop('im')

u'inner membrane'

In [34]:
model.optimize()

UnicodeEncodeError: 'ascii' codec can't encode character u'\xd7' in position 6397: ordinal not in range(128)

<Solution 0.088 at 0x1151ab250>

# Save Model

In [37]:
import cobra

target_filename_json = relative_directory + '/Reconstructions/MethylococcusModel15.json'
target_filename_xml = relative_directory + '/Reconstructions/MethylococcusModel15.xml'
cobra.io.write_sbml_model(model, target_filename_xml, use_fbc_package=True)
cobra.io.save_json_model(model, target_filename_json,pretty = True)

In [4]:
relative_directory = getcwd()
model = load_json_model(relative_directory + '/Reconstructions/MethylococcusModel15.json')